# Webcam setup test

In [ ]:
import cv2

In [ ]:
cap = cv2.VideoCapture(0)
cam_resolution = (1280, 720)

In [ ]:
while True: 
    _, frame = cap.read()
    image_np = cv2.resize(frame, cam_resolution)
    
    
    cv2.imshow("object detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
cv2.destroyAllWindows() 

In [ ]:
cap.release()

# NEW main

In [1]:
import cv2
import detect_handler as detect
from hoch_regal_lager import Hochregallager
from camera import MyCamera
import coordinates

In [2]:
hochregallager = Hochregallager()
cam_resolution = (1280, 720)

In [3]:
# provide MyCamera with correct cam source (ID), standard is 0
cam = MyCamera(0)
# print("ENTERED generate_frames!")
while True:
    # print("IN while True BLOCK!")
    if not cam.frame_available():
        continue
    # print("IN while True BLOCK! - IF ret WAS TRUE")

    image_np_with_detections = detect.handle_detection(camera=cam, hochregallager=hochregallager)
    # resize to match original resolution
    image_np_with_detections = cv2.resize(image_np_with_detections, cam_resolution)
    
    
    cv2.imshow("object detection", image_np_with_detections)
    
    # break with "q" key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam.release()
        break
cv2.destroyAllWindows() 
    
    

# Comparison between ARUCO based grid and Behaelter based grid

In [4]:
import cv2
import detect_handler as detect
from hoch_regal_lager import Hochregallager
from camera import MyCamera
import coordinates
import numpy as np

In [5]:
hochregallager = Hochregallager()
cam_resolution = (1280, 720)

In [6]:
def handle_aruco_corner(corners):
    ymin, xmin, ymax, xmax = corners[0][1], corners[0][0], corners[0][1], corners[0][0]
    
    for point in corners:
        if point[0] < xmin:
            xmin = point[0]
        if point[0] > xmax:
            xmax = point[0]
        if point[1] < ymin:
            ymin = point[1]
        if point[1] > ymax:
            ymax = point[1]
    
    return ymin, xmin, ymax, xmax

In [8]:
# provide MyCamera with correct cam source (ID), standard is 0
cam = MyCamera(0)
# print("ENTERED generate_frames!")
while True:
    # print("IN while True BLOCK!")
    if not cam.frame_available():
        continue
    # print("IN while True BLOCK! - IF ret WAS TRUE")
    ##################################################################
    tmp_image = np.array(cam.frame)
    # setup for marker detection
    aruco_params = cv2.aruco.DetectorParameters_create()
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    current_frame = tmp_image.copy()
    current_frame2 = tmp_image.copy()
    # gray scale should help improve the marker detection
    gray_img = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # detectMarkers returns  multiple objects
    aruco_bboxes, aruco_ids, _ = cv2.aruco.detectMarkers(gray_img, aruco_dict, parameters=aruco_params)
    # cv2.polylines(gray_img, np.int64(aruco_bboxes[0]), True, (0, 255, 100), 4)
    cv2.aruco.drawDetectedMarkers(tmp_image, aruco_bboxes)
    #print(aruco_ids)
    #print(aruco_bboxes[0][0])
    #print(aruco_bboxes[1])
    
    # only try if both markers are detected, else the following code will find the outline using detected behaelter
    if len(aruco_bboxes) == 2:
        
        b1_ymin, b1_xmin, b1_ymax, b1_xmax = handle_aruco_corner(aruco_bboxes[0][0])
        b2_ymin, b2_xmin, b2_ymax, b2_xmax = handle_aruco_corner(aruco_bboxes[1][0]) 
        
        if b1_xmin < b2_xmin:
            grid_xmin = b1_xmax
            grid_xmax = b2_xmin
            grid_ymin = b1_ymin
            grid_ymax = b2_ymin
        else:
            grid_xmin = b2_xmax
            grid_xmax = b1_xmin
            grid_ymin = b2_ymin
            grid_ymax = b1_ymin
        
        
        #grid_ymin = b1_ymin if b1_ymin < b2_ymin else b2_ymin
        #grid_xmin = b1_xmin if b1_xmin < b2_xmin else b2_xmin
        #grid_ymax = b1_ymax if b1_ymax > b2_ymax else b2_ymax
        #grid_xmax = b1_xmax if b1_xmax > b2_xmax else b2_xmax

        #pts = np.int64(((grid_xmin, grid_ymin),(grid_xmax, grid_ymax))
        #pts = np.int64(()([(grid_xmin, grid_ymin),(grid_xmax, grid_ymax)])
        pt1 = np.int64((grid_xmin, grid_ymin))
        pt2 = np.int64((grid_xmax, grid_ymax))
        color = (0, 255, 100)
        thickness = 4
        cv2.rectangle(tmp_image, pt1, pt2, color, thickness)
        percent= 1/3
        
        bottom = grid_ymin
        right = grid_xmin
        percent_move_x = (grid_xmax-grid_xmin)*percent
        percent_move_y = (grid_ymax-grid_ymin)*percent
        top = grid_ymin - percent_move_y
        left = grid_xmin -  percent_move_x
        
        for row in range(3):
            top = top + percent_move_y
            bottom = bottom + percent_move_y
            left, right = (grid_xmin - percent_move_x), grid_xmin
            for column in range(3):
                left = left + percent_move_x
                right = right + percent_move_x
                
                
                pt1 = np.int64((left, top))
                pt2 = np.int64((right, bottom)) 
                cv2.rectangle(tmp_image, pt1, pt2, color, thickness)
                
                
        #cv2.imshow('eg', tmp_image)
        
      
    image_np_with_detections = detect.handle_detection(camera=cam, hochregallager=hochregallager) 
    
    behaelter_list = hochregallager.behaelter_obj_list
    ymin_test, xmin_test, ymax_test, xmax_test = behaelter_list[0].bounding_box[0], behaelter_list[
        0].bounding_box[1], behaelter_list[0].bounding_box[2], behaelter_list[0].bounding_box[3]

    for behaelter in behaelter_list:
        new_ymin, new_xmin, new_ymax, new_xmax = behaelter.bounding_box
        if ymin_test > new_ymin:
            ymin_test = new_ymin
        if xmin_test > new_xmin:
            xmin_test = new_xmin
        if ymax_test < new_ymax:
            ymax_test = new_ymax
        if xmax_test < new_xmax:
            xmax_test = new_xmax
    #return (ymin_test, xmin_test, ymax_test, xmax_test)
    bottom = ymin_test
    right = xmin_test
    percent_move_x = (xmax_test-xmin_test)*percent
    percent_move_y = (ymax_test-ymin_test)*percent
    top = ymin_test - percent_move_y
    left = xmin_test -  percent_move_x

    for row in range(3):
        top = top + percent_move_y
        bottom = bottom + percent_move_y
        left, right = (xmin_test - percent_move_x), xmin_test
        for column in range(3):
            left = left + percent_move_x
            right = right + percent_move_x


            pt1 = np.int64((left, top))
            pt2 = np.int64((right, bottom)) 
            cv2.rectangle(current_frame, pt1, pt2, color, thickness)
            
    ##
    #modified
    if grid_ymin > ymin_test:
        third_ymin = ymin_test
        third_xmin = grid_xmin
        third_ymax = grid_ymax
        third_xmax = grid_xmax
    
        bottom = third_ymin
        right = third_xmin
        percent_move_x = (third_xmax-third_xmin)*percent
        percent_move_y = (third_ymax-third_ymin)*percent
        top = third_ymin - percent_move_y
        left = third_xmin -  percent_move_x

        for row in range(3):
            top = top + percent_move_y
            bottom = bottom + percent_move_y
            left, right = (third_xmin - percent_move_x), third_xmin
            for column in range(3):
                left = left + percent_move_x
                right = right + percent_move_x


                pt1 = np.int64((left, top))
                pt2 = np.int64((right, bottom)) 
                cv2.rectangle(image_np_with_detections, pt1, pt2, color, thickness)

        cv2.imshow('MIXED', image_np_with_detections)
    
        
    ##
            
    cv2.imshow('BEHAELTER', tmp_image)
    cv2.imshow('ARUCO', current_frame)    
        
    ##################################################################
    #image_np_with_detections = detect.handle_detection(camera=cam, hochregallager=hochregallager)
    # resize to match original resolution
    #image_np_with_detections = cv2.resize(image_np_with_detections, cam_resolution)
    #cv2.imshow("object detection", tmp_image)
    
    # break with "q" key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam.release()
        break
cv2.destroyAllWindows() 